In [ ]:
!wget https://github.com/facebookresearch/DrQA/archive/master.zip
!unzip master.zip

In [ ]:
!wget https://s3-eu-west-1.amazonaws.com/fever.public/train.jsonl

In [6]:
!mv DrQA-master/* .

In [ ]:
!python3 setup.py develop
!pip install -r requirements.txt

In [3]:
from drqa import retriever

In [ ]:
!wget https://github.com/offendo/best-words/raw/master/project_data/wiki_docs_skimmed-tfidf-ngram%3D2-hash%3D16777216-tokenizer%3Dsimple.npz.zip
!wget https://github.com/offendo/best-words/raw/master/project_data/wiki_docs_skimmed.db.zip

!unzip wiki_docs_skimmed-tfidf-ngram=2-hash=16777216-tokenizer=simple.npz.zip
!unzip wiki_docs_skimmed.db.zip

In [8]:
import json
import itertools
with open('train.jsonl', 'r') as fp:
    claims = []
    for jsonl in fp.readlines():
        jsoned = json.loads(jsonl)
        evidence_set = set()
        for evidence in itertools.chain(*jsoned['evidence']):
            if evidence[2] is not None:
                evidence_set.add(evidence[2])
        claims.append((jsoned['claim'], evidence_set))

In [13]:
model_path = 'wiki_docs_skimmed-tfidf-ngram=2-hash=16777216-tokenizer=simple.npz'
ranker = retriever.get_class('tfidf')(tfidf_path=model_path)

In [19]:
predicted = []
for claim, _ in claims:
    closest_docs = set(ranker.closest_docs(claim, k=5)[0])
    predicted.append((claim, closest_docs))

In [33]:
tp = 0
p = 0
for (_, gold), (_, pred) in zip(claims, predicted):
    p += len(gold)
    tp += len(gold & pred)

print('Total Recall of tf-idf: {}%'.format(round(100 * (tp/p), 2)))

Total Recall of tf-idf: 77.1%
